Kyle Mckay - F21DL Coursework Portfolio
===

1. [Data Set Choice](#h1)
2. [Visualization and Initial Data Exploration](#h2)
    1. [Class Distribution](#h2_0)
    1. [Direct Comparison](#h2_1)
    2. [Conversion to Pixel Data](#h2_2)
    3. [Average Images](#h2_3)
    4. [Standard Deviation](#h2_4)
3. [Acknowledgement](#ack)

# Setup

In [1]:
%matplotlib inline
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Initalise variables and functions used throughout the notebook

In [2]:
data_dir = 'DATA' 
img_dir = f'{data_dir}/images'

# Utility method to plot numpy greyscale pixel matrices side by side for comparison
def multi_plot(*imgs):
    fig = plt.figure()

    # TODO add multiple rows if many images
    for i, avg in enumerate(imgs, 1):
        title, dat = avg
        ax = fig.add_subplot(1, len(imgs), i)
        ax.imshow(dat, vmin=0, vmax=255, cmap='Greys_r')
        plt.title(title)
        plt.axis('off')
    
    plt.show()

# Functional approach to collapsing 3rd order numpy greyscale pixel tensors using certain methods (e.g. mean)
def collapse_img(full_mat, fn = np.mean, size = (300, 180)):
    # Apply desired function
    collapsed_img = fn(full_mat, axis = 0)
    
    # Reshap back to a matrix
    return collapsed_img.reshape(size)

# <a id="h1">Data Set Choice</a>

I chose to work with the FGVC-Aircraft data set (Maji et al.) because:

- It is an image dataset and I'm interested in computer vision and image processing techniques.
- The data set is intended for use as a multiclass classification problem, which provides an interesting level of complexity above binary classification and below multi-label classification.
- There is granularity to the classes in the data set. With 41 manufacturers providing plenty of instances (image files) to work with for each.
- The images are not provided in a uniform size or aspect ratio, which adds some complexity to homogenising and reducing the data in preprocessing to meet computational constraints.
- The data provenance is clearly stated and acceptable for my research/learning purposes.

# <a id="h2">Visualization and Initial Data Exploration</a>

## <a id="h2_0">Class Distribution</a>

I first want to get a sense for the number of instances there are per manufacturer since I know there are 100 images per aircraft model (the highest level of granularity), but have no sense for how evenly distributed those are among the lower class granularity levels.

This is important since for training purposes I want my data to have an approximately even distribution to avoid biasing classification results.

In [3]:
# Data set provides text files listing the label that applies to each image
img_labels = pd.read_csv(
    f'{data_dir}/images_manufacturer_train.txt',
    sep=' ', header=None, 
    names=['Image', 'Class'],
    skip_blank_lines=True,
)

# Interested in classes with at least 100 instances (reasonable quantity)
class_counts = img_labels['Class'].value_counts()
class_counts = class_counts[class_counts >= 100]
class_counts


Boeing       733
Airbus       434
Embraer      233
McDonnell    232
de           167
Canadair     134
Cessna       133
British      133
Douglas      133
Lockheed     102
Fokker       100
Name: Class, dtype: int64

Evidently there is not an even distribution of the manufacturer classes. TODO: Drop classes, drop images?


## Image Size Distribution

I next want to get a sense for how homogeneous the images are in terms of their dimensions. This is relevant to check if preprocessing needs to be done and, if so, which approach I will take.

In [6]:
# Only interested in images with sufficient number of class instances
all_imgs = img_labels[img_labels['Class'].isin(class_counts.keys())]['Image']


sizes = []
for fn in all_imgs:
    # File names are padded with 0 to 7 digits
    img = cv2.imread(f'{img_dir}/{fn:07}.jpg')

    # Skip missing files for now
    if img is None:
        continue

    sizes.append(img.shape[:2])
sizes = pd.DataFrame(sizes, columns=['Height', 'Width'])

sizes.describe()

,Height,Width
count,2534.000000,2534.000000
mean,751.729282,1105.763220
std,116.229313,168.916645
min,434.000000,775.000000
25%,693.000000,1024.000000
50%,700.000000,1024.000000
75%,812.000000,1200.000000
max,1188.000000,1600.000000


I can see from the dimension descriptions that the images vary in size by a factor of ~2 in width and ~3 in height. They will need to be homogenised to a standard size so that the features (pixels) corrolate for later processing.

Thankfully, the data set provides bounding boxes for the aircraft in each image, so I next want to find the minimum and maximum dimensions for those as I can use them to determine how I will approach preprocessing these images to standard dimensions.


## <a id="h2_1">Direct Comparison</a>

The following code pulls 3 random sample images from each class for an initial direct visual comparison.

It's clear from a quick visual inspection that the pollen carrying bees have distinct features (the pollen) on either side of their abdomen which isn't present in the other images. This suggests to me it should be fine to process the images in greyscale and reduce the number of features for efficiency since they should still be distinguishable.

## <a id="h2_2">Conversion to Pixel Data</a>

The following script is converting greyscale versions of the image files into a 3rd order tensor with $n\times m$ elements. $n$ is the number of observations and $m$ is the number of pixels (features). The element values represent the pixel shade.

## <a id="h2_3">Average Images</a>

By averaging the pixel data for all of a given class, I can get a sense for the regions which are common to each class. It's not as clear as before, but there's still an obvious presence on either side of the abdomen for the pollen carrying class which isn't present in the other images.

### Average Images Difference

The difference between the classes becomes clearer by taking the difference of these two averaged images. Now it's much more obvious that one class (pollen carrier) has a distinct region of pixels in the lower left and a less distinct region in the lower right.

## <a id="h2_4">Standard Deviation</a>

To get a sense of the pixel regions which vary the most in each class, the same reasoning of averaging can be applied, but instead taking the standard deviation. The resulting images aren't as clear, but here I can see a lot more varience in those lower left and right regions for the pollen carrying class. This reinforces my understanding of the data.

# <a id="ack">Acknowledgement</a>

FGVC-Aircraft data set:

Fine-Grained Visual Classification of Aircraft, S. Maji, J. Kannala, E. Rahtu, M. Blaschko, A. Vedaldi, [arXiv.org](https://arxiv.org/abs/1306.5151), 2013

Image Data Exploration Code adapted from: [https://towardsdatascience.com/exploratory-data-analysis-ideas-for-image-classification-d3fc6bbfb2d2](https://towardsdatascience.com/exploratory-data-analysis-ideas-for-image-classification-d3fc6bbfb2d2)